In [1]:
from pathlib import Path
import pandas as pd

In [4]:
home_dir = Path.home()
inside_airbnb_data_dir = home_dir / 'Programming/data/inside-airbnb/london'
mps_monthly_crime_data_dir = home_dir / 'Programming/data/london-datastore/mps-monthly-crime-dashboard-data'

In [7]:
inside_airbnb_data_file = inside_airbnb_data_dir / 'listings.csv'
mps_monthly_crime_data_file = mps_monthly_crime_data_dir / 'M1045_MonthlyCrimeDashboard_TNOCrimeData.xlsx'

In [32]:
mps_monthly_crime_df = pd.read_excel(mps_monthly_crime_data_file, parse_dates=['Month_Year'],
                                     usecols=['Month_Year', 'Borough_SNT', 'Count'],
                                     index_col=0, skiprows=range(1, 400000))

In [33]:
mps_monthly_crime_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 389527 entries, 2023-08-01 to 2025-02-01
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Borough_SNT  389527 non-null  object
 1   Count        389527 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.9+ MB


In [40]:
columns_list = ['neighbourhood_cleansed', 'bathrooms', 'bedrooms', 'latitude', 'longitude',
                'room_type', 'latitude', 'longitude', 'property_type', 'price']
inside_airbnb_df = pd.read_csv(inside_airbnb_data_file, usecols=columns_list)
inside_airbnb_df.head()

,neighbourhood_cleansed,latitude,longitude,property_type,room_type,bathrooms,bedrooms,price
0,Hammersmith and Fulham,51.49392,-0.22754,Entire rental unit,Entire home/apt,1.5,1.0,$89.00
1,Tower Hamlets,51.52435,-0.06938,Entire condo,Entire home/apt,3.0,3.0,$220.00
2,Hammersmith and Fulham,51.49547,-0.22864,Entire rental unit,Entire home/apt,1.0,1.0,$88.00
3,Hammersmith and Fulham,51.49368,-0.22774,Room in aparthotel,Private room,1.0,1.0,$75.00
4,Haringey,51.59031,-0.09408,Private room in rental unit,Private room,1.0,1.0,$53.00


In [35]:
inside_airbnb_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca